# Writing to Clever Cloud PostGIS db

In [1]:
import psycopg2 as pg
from psycopg2 import sql
import pandas as pd
import os
# from IPython.display import Markdown as md
import plotly.express as px

In [2]:
db_host = os.environ.get('DB_HOST_CLOUD')
db_port = os.environ.get('DB_PORT_CLOUD')
db_user = os.environ.get('DB_USER_CLOUD')
db_password = os.environ.get('DB_PASSWORD_CLOUD')
db_name = os.environ.get('DB_NAME_CLOUD')

db_url = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

# Adding data into PostGres database on Clever Cloud

In [3]:
# #create table from dataframe - Clever Cloud (Execution time ~= 3 min)
# import pandas as pd
# # read the csv file into a pandas dataframe, replacing NaN with None
# df = pd.read_csv('final_output.csv', index_col=['cep_id'], dtype={'pa': 'Int64', 'eco': 'Int64', 'country': 'Int64'}).replace({pd.NA: None})
# from sqlalchemy import create_engine
# engine = create_engine(db_url)
# df.to_sql('cep_water', engine, if_exists='replace')

# Grouping transition bands into view
- seasonal at start of time series (bands: 4, 6 and 7)
- seasonal at end of time series (bands: 1, 3 and 8)
- permanent at start of time series (bands: 4, 5 and 8)
- permanent at end of time series (bands: 1, 2 and 7)

In [4]:
# get the column names
table_columns =  pd.read_sql_query('SELECT * FROM cep_water LIMIT 0', db_url).columns
# remove all the transition columns
table_columns = [col for col in table_columns if 'transition' not in col]

# manually create the column names for the different categories
seasonal_at_start_columns = table_columns #+ ['transition_4', 'transition_6', 'transition_7']
seasonal_at_end_columns = table_columns + ['transition_1', 'transition_3', 'transition_8']
permanent_at_start_columns = table_columns + ['transition_4', 'transition_5', 'transition_8']
permanent_at_end_columns = table_columns + ['transition_1', 'transition_2', 'transition_7']

In [5]:
# connect to the database
conn = pg.connect(
    database=db_name,
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)

# create a cursor
cur = conn.cursor()

# select all columsn but sum the transitions fields for 3,6,7.
query = f'''
CREATE VIEW seasonal_1984 AS
SELECT 'cep_id', 'pa', 'eco',
SUM(transition_4 + transition_6 + transition_7) AS seasonal_area
FROM cep_water
'''

# execute the query and fetch the result
cur.execute(query)

# read view into a pandas dataframe
#df = pd.read_sql_query('SELECT * FROM seasonal_1984', db_url)

# execute the query
# cur.execute(query)

cur.close()
conn.close()            

DuplicateColumn: column "?column?" specified more than once


In [8]:
cur.close()
conn.close()      

In [ ]:
query = f'''
DROP VIEW IF EXISTS "seasonal_perma_groupings";
CREATE VIEW seasonal_perma_groupings AS
WITH groupings AS (
    SELECT 
		cep_water.cep_id,
		SUM(transition_4 + transition_6 + transition_7) AS seasonal_1984,
		SUM(transition_1 + transition_3 + transition_8) AS seasonal_2015,
		SUM(transition_4 + transition_5 + transition_8) AS permanent_1984,
		SUM(transition_1 + transition_2 + transition_7) AS permanent_2015
	FROM 
		cep_water 
	GROUP BY 
		cep_water.cep_id
)
-- join temp groupings table with cep_water table, but don't select transition columns
SELECT 
    cep_water.cep_id,
    country,
    country_name,
    iso3,
    eco,
    eco_name,
    is_marine,
    pa,
    pa_name,
    is_protected,
    groupings.seasonal_1984,
    groupings.seasonal_2015,
    groupings.permanent_1984,
    groupings.permanent_2015
FROM
    cep_water
JOIN
    groupings
ON
    cep_water.cep_id = groupings.cep_id;
'''
print(query)


CREATE VIEW seasonal_1984 AS
SELECT cep_id, country, country_name, iso3, eco, eco_name, is_marine, pa, pa_name, is_protected,
SUM(transition_4 + transition_6 + transition_7) AS seasonal_area
FROM cep_water
GROUP BY cep_id, country, country_name, iso3, eco, eco_name, is_marine, pa, pa_name, is_protected



In [ ]:
# connect to the database
conn = pg.connect(
    database=db_name,
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)

# create a cursor
cur = conn.cursor()
#check if the view was created
cur.execute('SELECT * FROM seasonal_1984')
rows = cur.fetchall()
for row in rows:
    print(row)
    break



# #create a view containing just cep_id 
# cur = conn.cursor()
# query = f'''
# CREATE OR REPLACE VIEW my_view AS
# SELECT cep_id
# FROM cep_water
# '''
# cur.execute(query)
# # read the data from the view
# cur.execute('SELECT * FROM my_view')

# rows = cur.fetchall()
# for row in rows:
#     print(row)
cur.close()
conn.close()


UndefinedTable: relation "seasonal_1984" does not exist
LINE 1: SELECT * FROM seasonal_1984
                      ^


In [ ]:
raise

In [9]:
# connect to the database
conn = pg.connect(
    database=db_name,
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)

In [10]:
# check size of database
cursor = conn.cursor()
cursor.execute('SELECT * FROM cep_water LIMIT 1')
result = cursor.fetchall()
for row in result:
    print(row)
cursor.close()

(1, 895.792133, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 171, 'Lithuania', 'LTU', 80412, 'Central European mixed forests', False, 0, None, False)


# Data Completeness & Consistency checks
- missing values (done)
- duplicates (done)
- data types (done)
- data ranges
- is marine (all permanent water bodies are marine)
### case studies https://www.nature.org/content/dam/tnc/nature/en/documents/Pathway_for_Inland_Waters_Nov_2022.pdf

In [11]:
# count number of rows in database table
cursor = conn.cursor()
cursor.execute('SELECT COUNT(*) FROM cep_water')
db_length = cursor.fetchone()[0]
cursor.close()

In [12]:
# check for any missing country (country = 0)
cursor = conn.cursor()
cursor.execute('SELECT * FROM cep_water WHERE country = 0')
result = cursor.fetchall()
for row in result:
    print(row)
cursor.close()

In [13]:
# check for duplicates
cursor = conn.cursor()
# check for duplicate cep_id and pa and eco
cursor.execute('SELECT cep_id, pa, eco, COUNT(*) FROM cep_water GROUP BY cep_id, pa, eco HAVING COUNT(*) > 1')
result = cursor.fetchall()
for row in result:
    print(row)
cursor.close()

### Checking if Marine only areas have no permanent water bodies 
#### Findings: some marine only areas only 

In [14]:
# check where is_marine true, it has large band 1 area the other values should be 0 or near 0 if it's near coastlines (e.g reefs)
cursor = conn.cursor()
# Select all rows where is_marine is true and transition_1 is less than 1
cursor.execute('SELECT * FROM cep_water WHERE is_marine = TRUE AND "transition_1" < 1')
# put the result in a dataframe and add the column names
df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
df.set_index('cep_id', inplace=True)
cursor.close()
percentage_of_marine_without_permanent_water = len(df) / db_length * 100
df.head()

,transition_0,transition_1,transition_2,transition_3,transition_4,transition_5,transition_6,transition_7,transition_8,transition_9,transition_10,country,country_name,iso3,eco,eco_name,is_marine,pa,pa_name,is_protected
cep_id,,,,,,,,,,,,,,,,,,,,
316,78844.647179,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,6,Albania,ALB,20030,Adriatic Sea,True,11664,Rrushkull,True
548,17413.521867,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,696.524314,8,United Arab Emirates,ARE,20090,Arabian (Persian) Gulf,True,555622078,Al Zorah,True
553,43883.138540,0.0,0.0,0.0,0.0,1393.109387,0.0,0.0,0.0,0.0,13234.579442,8,United Arab Emirates,ARE,20090,Arabian (Persian) Gulf,True,555625658,Al Zorah,True
658,3113.515126,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,5604.311130,9,Argentina,ARG,20182,Rio de la Plata,True,19604,Bahía de Samborombón,True
658,3113.515126,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,5604.311130,9,Argentina,ARG,20182,Rio de la Plata,True,555577543,Campos del Tuyú,True


In [15]:
# check where is_marine true, it has large band 1 area the other values should be 0 or near 0 if it's near coastlines (e.g reefs)
cursor = conn.cursor()
# Select all rows where is_marine is true and transition_1 is less than 1
cursor.execute('SELECT * FROM cep_water WHERE is_marine = TRUE AND "transition_1" > 1')
# put the result in a dataframe and add the column names
df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
df.set_index('cep_id', inplace=True)
cursor.close()
percentage_of_marine_with_permanent_water = len(df) / db_length * 100
df.head()

,transition_0,transition_1,transition_2,transition_3,transition_4,transition_5,transition_6,transition_7,transition_8,transition_9,transition_10,country,country_name,iso3,eco,eco_name,is_marine,pa,pa_name,is_protected
cep_id,,,,,,,,,,,,,,,,,,,,
2,1.366843e+12,7.194092e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Area Beyond National Jurisdiction,ABNJ,1,Antarctic,True,0,None,False
4,0.000000e+00,1.836981e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Area Beyond National Jurisdiction,ABNJ,1,Antarctic,True,555547601,South Georgia and South Sandwich Islands Marin...,True
6,6.328776e+11,1.038784e+13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Area Beyond National Jurisdiction,ABNJ,2,Antarctic Polar Front,True,0,None,False
7,0.000000e+00,1.167926e+10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Area Beyond National Jurisdiction,ABNJ,2,Antarctic Polar Front,True,345888,Terres Australes Françaises,True
8,0.000000e+00,5.916302e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Area Beyond National Jurisdiction,ABNJ,2,Antarctic Polar Front,True,345888,Terres Australes Françaises,True


In [16]:
fig = px.pie(values=[percentage_of_marine_without_permanent_water, percentage_of_marine_with_permanent_water, 100 - percentage_of_marine_without_permanent_water - percentage_of_marine_with_permanent_water], names=['Marine with no permanent water', 'Marine with permanent water', 'Non-marine area (any bands)'], title='Percentage of marine areas with and without permanent water')
# change layout to dark theme make the chart square
fig.update_layout(template='plotly_dark', width=600, height=600)
fig.show()

In [17]:
conn.close()

In [ ]:
raise Exception('Stop here')

# Summaries

In [ ]:
def get_summed_bands_by_col(conn, group_by_col = "country_name",is_pa=False, is_marine=False, convert_to_meters=False, ):
    try:
        # create a cursor object using the cursor() method
        cursor = conn.cursor()
        # query all rows transition_0  to transition_10, filter where pa is 0 and where marine is false, and group by country_name and sum the areas for each transition band
        bands_SUM = [f'SUM(transition_{i}) as "transition_{i}"' for i in range(11)]
        query = sql.SQL('SELECT {group_by_column}, {bands} FROM cep_water WHERE is_protected = {is_pa} AND is_marine = {is_marine} GROUP BY {group_by_column}').format(
            bands=sql.SQL(', ').join(map(sql.SQL, bands_SUM)),
            is_pa=sql.Literal(is_pa),
            is_marine=sql.Literal(is_marine),
            group_by_column=sql.Identifier(group_by_col)
        )
        # #print query as string
        # print(query.as_string(conn))                                                                                                                    
        # filter where 
        cursor.execute(query)
        result = cursor.fetchall()
        cursor.close()

        # convert to df and set index to group_by_col
        df = pd.DataFrame(result, columns=[group_by_col]+[f'transition_{i}' for i in range(11)])
        df.set_index(group_by_col, inplace=True)
        # convert to meters
        if convert_to_meters:
            df = df.apply(lambda x: x/1000000)
        return df
        
        #return result
    except Exception as e:
        print("error: ", e)
    finally:
        conn.rollback()

In [ ]:
df_protected = get_summed_bands_by_col(conn, group_by_col = "country_name",is_pa=True, is_marine=False, convert_to_meters=True)
df_unprotected = get_summed_bands_by_col(conn, group_by_col = "country_name",is_pa=False, is_marine=False, convert_to_meters=True)
df_terrestrial_eco = get_summed_bands_by_col(conn, group_by_col = "eco",is_pa=False, is_marine=False, convert_to_meters=True)

In [ ]:
df_terrestrial_eco

In [ ]:
df_protected

In [ ]:
df_unprotected

In [ ]:
conn.close()